# Подключаем библиотеки и загружаем данные

In [1]:
import pandas as pd

In [2]:

import numpy as np
from catboost import CatBoostClassifier
import optuna
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

c:\Users\Venya\OneDrive\Документы\Универ\Магистратура\2_семестр\Big_data\Лаб_1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import os
import glob
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
parent_directory

'c:\\Users\\Venya\\OneDrive\\Документы\\Универ\\Магистратура\\2_семестр\\Big_data\\Лаб_1'

In [ ]:
friday_ddos = pd.read_csv(os.path.join(parent_directory, "data//Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"), sep=",")
friday_portscan = pd.read_csv(os.path.join(parent_directory,"data//Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"), sep=",")

In [28]:
thursday_inf = pd.read_csv(os.path.join(parent_directory,"data//Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"), sep=",")


In [29]:
import pandas as pd
import os
import chardet

# Путь к файлу
file_path = os.path.join(parent_directory, "data", "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")

# Определение кодировки
with open(file_path, "rb") as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result["encoding"]
    confidence = result["confidence"]

print(f"Определённая кодировка: {encoding} (уверенность: {confidence})")

# Чтение файла с определённой кодировкой
try:
    thursday_webattack = pd.read_csv(file_path, sep=",", encoding=encoding)
    print(thursday_webattack.head())
except UnicodeDecodeError:
    print("Не удалось прочитать файл с определённой кодировкой. Попробуйте другую кодировку.")

Определённая кодировка: Windows-1252 (уверенность: 0.73)


C:\Users\Venya\AppData\Local\Temp\ipykernel_19640\675830202.py:19: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  thursday_webattack = pd.read_csv(file_path, sep=",", encoding=encoding)


                                  Flow ID      Source IP   Source Port  \
0  192.168.10.3-192.168.10.50-389-33898-6  192.168.10.50       33898.0   
1  192.168.10.3-192.168.10.50-389-33904-6  192.168.10.50       33904.0   
2                   8.0.6.4-8.6.0.1-0-0-0        8.6.0.1           0.0   
3  192.168.10.14-65.55.44.109-59135-443-6  192.168.10.14       59135.0   
4  192.168.10.3-192.168.10.14-53-59555-17  192.168.10.14       59555.0   

   Destination IP   Destination Port   Protocol      Timestamp  \
0    192.168.10.3              389.0        6.0  6/7/2017 8:59   
1    192.168.10.3              389.0        6.0  6/7/2017 8:59   
2         8.0.6.4                0.0        0.0  6/7/2017 8:59   
3    65.55.44.109              443.0        6.0  6/7/2017 8:59   
4    192.168.10.3               53.0       17.0  6/7/2017 8:59   

    Flow Duration   Total Fwd Packets   Total Backward Packets  ...  \
0     113095465.0                48.0                     24.0  ...   
1     113473706.

In [30]:
thursday_webattack

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.3-192.168.10.50-389-33898-6,192.168.10.50,33898.0,192.168.10.3,389.0,6.0,6/7/2017 8:59,113095465.0,48.0,24.0,...,32.0,203985.500,5.758373e+05,1629110.0,379.0,13800000.0,4.277541e+06,16500000.0,6737603.0,BENIGN
1,192.168.10.3-192.168.10.50-389-33904-6,192.168.10.50,33904.0,192.168.10.3,389.0,6.0,6/7/2017 8:59,113473706.0,68.0,40.0,...,32.0,178326.875,5.034269e+05,1424245.0,325.0,13800000.0,4.229413e+06,16500000.0,6945512.0,BENIGN
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0.0,8.0.6.4,0.0,0.0,6/7/2017 8:59,119945515.0,150.0,0.0,...,0.0,6909777.333,1.170000e+07,20400000.0,6.0,24400000.0,2.430000e+07,60100000.0,5702188.0,BENIGN
3,192.168.10.14-65.55.44.109-59135-443-6,192.168.10.14,59135.0,65.55.44.109,443.0,6.0,6/7/2017 8:59,60261928.0,9.0,7.0,...,20.0,0.000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.14-53-59555-17,192.168.10.14,59555.0,192.168.10.3,53.0,17.0,6/7/2017 8:59,269.0,2.0,2.0,...,32.0,0.000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Если в наборе данных несколько типов аномалий, столбец с метками может содержать строки (например, 'BENIGN', 'DoS', 'DDoS', и т.п.)
# Преобразуем метки в числовой формат
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])

# Отделяем признаки от целевой переменной
X = data.drop(columns=['Label'])
y = data['Label']

# Разбиваем данные на обучающую и тестовую выборки (20% на тест)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Подбираем параметры для классификатора

In [ ]:
def objective(trial):
    # Определяем пространство гиперпараметров
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_uniform('random_strength', 1e-9, 10.0),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
        'verbose': 0,  # отключаем вывод обучения
        # Если классификация мультиклассовая, выбираем соответствующую функцию потерь
        'loss_function': 'MultiClass' if len(np.unique(y)) > 2 else 'Logloss'
    }
    
    # Инициализируем модель с подобранными гиперпараметрами
    model = CatBoostClassifier(**params, random_state=42)
    
    # Используем StratifiedKFold для кросс-валидации (3 фолда)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, valid_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50, verbose=False)
        preds = model.predict(X_val)
        acc = accuracy_score(y_val, preds)
        scores.append(acc)
    return np.mean(scores)

# Обучаем CatBoost используя Optuna 

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # число итераций можно увеличить для лучшего поиска

print("Лучшая оптимизация:")
best_trial = study.best_trial
print("  Лучший результат (accuracy):", best_trial.value)
print("  Лучшие гиперпараметры:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
best_params = best_trial.params.copy()
best_params.update({
    'verbose': 0,
    'loss_function': 'MultiClass' if len(np.unique(y)) > 2 else 'Logloss',
    'random_state': 42
})

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train, y_train)

# Оценка модели на тестовой выборке

In [ ]:
y_pred = final_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("\nТочность на тестовой выборке: {:.4f}".format(accuracy))

print("\nОтчёт по классификации:")
print(classification_report(y_test, y_pred, target_names=le.classes_.astype(str)))

print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))